# 1. Naver API를 이용한 블로그 포스트들의 URL 받아오기


- 현재로서는 가져올 수 있는 블로그 포스트 링크의 개수가 최대 1100개로 제한되어 있는 것으로 보임.<br/>(이게 걸림돌로 작용될 수 있을듯..)
- API를 이용해서 데이터를 호출할 수 있는 횟수가 하루에 2만 5천번으로 제한되어 있음.
- 현재 이태우의 API 키와 비밀번호를 이용하여 코드가 작성되어 있음 (API_PARAMETER)
- 현재는 많은 링크를 가져올 필요가 없으므로, ### 테스트를 위해 넣은 부분 ### 코드블록을 이용하여 API 호출 횟수를 제한함
<br/><br/>

In [29]:
# <Code Reference> 
# - (Github, xotrs/naver-blog-crawler) https://github.com/xotrs/naver-blog-crawler/blob/master/NaverBlogCrawler.py
# - (검색 API) https://developers.naver.com/docs/search/blog/

import os
import sys
import urllib.request
import json
import math

API_PARAMETER = {
    "client_id": "E5cL2__bFGNST8f4eknt",
    "client_secret": "kcwtUqHT98"
}

QUERY_PARAMETER = {
    "query": "네이버 클로바",
    "display": 100,
    "sort_type": "sim"
}

def naver_api_blog_search(api_parameter, search_word, 
                          display_count=10, start=1, sort_type='sim'):
    result = None
    
    client_id = api_parameter['client_id']
    client_secret = api_parameter['client_secret']
    
    encText = urllib.parse.quote("{}".format(search_word))
    url = "https://openapi.naver.com/v1/search/blog?query=" + encText + "&display=" + str(
            display_count) + "&start=" + str(start) + "&sort=" + sort_type  # json 결과
    # url = "https://openapi.naver.com/v1/search/blog.xml?query=" + encText # xml 결과 예시
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)
    response = urllib.request.urlopen(request)
    rescode = response.getcode()
    if(rescode==200):
        response_body = response.read()
        result = response_body.decode('utf-8')
    else:
        print("Error Code:" + rescode) 
    return result

def get_search_result_page_count(api_parameter, query_word, display_count):
    search_result_page_count = 0
    
    decoded_response_body = naver_api_blog_search(api_parameter, query_word)
    if decoded_response_body:
        json_response_body = json.loads(decoded_response_body)
        
        total_posts = json_response_body['total']
        if total_posts != 0:
            search_result_page_count = math.ceil(total_posts / display_count)
    return search_result_page_count
    
def get_urls(query_word, display_count, sort_type, search_result_page_count):
    url_list = []
    for i in range(search_result_page_count):
        start = display_count*i + 1
        if start > 1000:
            start = 1000
            
        decoded_response_body = naver_api_blog_search(api_parameter, query_word, display_count, start, sort_type)
        if decoded_response_body:
            json_response_body = json.loads(decoded_response_body)
            
            for dp in json_response_body['items']:
                url_list.append(dp['link'].replace("amp;", ""))
        
        ### 테스트를 위해 넣은 부분 ###
        #if i == 1:
        #    break
        #########################
        
        if start == 1000:
            break
    return url_list


if __name__ == '__main__':
    api_parameter = API_PARAMETER 
    query = QUERY_PARAMETER['query']
    display = QUERY_PARAMETER['display']
    sort_type = QUERY_PARAMETER['sort_type']
    
    p_count = get_search_result_page_count(api_parameter, query, display)
    urls = get_urls(query, display, sort_type, p_count)

In [30]:
urls

['https://blog.naver.com/sky121400?Redirect=Log&logNo=221493275584',
 'https://blog.naver.com/danjuck?Redirect=Log&logNo=221502038209',
 'https://blog.naver.com/ralrapo?Redirect=Log&logNo=221418872833',
 'https://blog.naver.com/yarosiku?Redirect=Log&logNo=221410642877',
 'https://blog.naver.com/ukb0914?Redirect=Log&logNo=221355525241',
 'https://blog.naver.com/aimei96?Redirect=Log&logNo=221288615932',
 'https://blog.naver.com/three486?Redirect=Log&logNo=221492447164',
 'https://blog.naver.com/iffu?Redirect=Log&logNo=221015291581',
 'https://blog.naver.com/lopil22?Redirect=Log&logNo=221412000291',
 'https://blog.naver.com/bless_yuna?Redirect=Log&logNo=221416528704',
 'https://blog.naver.com/soeun3093?Redirect=Log&logNo=221351131297',
 'https://blog.naver.com/ddiamosho?Redirect=Log&logNo=221408937972',
 'https://blog.naver.com/hdgyung?Redirect=Log&logNo=221435823424',
 'https://blog.naver.com/tw01554?Redirect=Log&logNo=221376679142',
 'https://blog.naver.com/ralrapo?Redirect=Log&logNo=22

In [31]:
len(urls)

1100

<br/><br/>
# 2. 받아온 URL들을 이용하여 원하는 정보를 scraping

In [3]:
# <Code Reference> 
# - (zilu11의 블로그 포스트) https://blog.naver.com/zilu11/221255183159

import requests
from bs4 import BeautifulSoup

import time
import datetime
import re


def get_realdata_url_naverblog(url):
    """
    네이버 블로그 포스트의 실제 데이터가 들어있는 URL을 반환해줌.
    보통 3가지 경우가 있는 것 같음.
    <지금까지 찾은 경우의 수>
    1. id='screenFrame'이 담긴 페이지에서 얻은 URL -> (base_url) + (id='mainFrame'이 담긴 페이지에서 얻은 URL) -> 실제 데이터가 담긴 URL
    2. (base_url) + (id='mainFrame'이 담긴 페이지에서 얻은 URL) -> 실제 데이터가 담긴 URL
    3. 검색해서 처음 얻은 URL이 실제 데이터가 들어가 있는 URL임
    """
    base_url = "http://blog.naver.com"
    
    r = requests.get(url)
    soup = BeautifulSoup(r.text.encode('utf-8'), 'html5lib')
    
    if soup.find(id='screenFrame'):
        # id = 'screenFrame'이 존재하는 경우 (id: screenFrame URL에서 -> id: mainFrame URL로 넘김)
        url = soup.find(id='screenFrame').get('src')
        r = requests.get(url)
        soup = BeautifulSoup(r.text.encode('utf-8'), 'html5lib')
    if soup.find(id='mainFrame'):
        # id = 'mainFrame'이 존재하는 경우 (id: mainFrame URL에서 -> id: real data URL로 넘김)
        url = base_url + soup.find(id='mainFrame').get('src')
    return url 

def get_title_naverblog(soup):
    """
    BeautifulSoup 객체를 받아서 블로그 포스트의 제목을 반환해줌.
    """
    try:
        #블로그 포스트 제목 추출
        title = soup.find('meta', attrs={'property':'og:title'}).get('content')
    except:
        title = 'NA'
    return title

def det_publish_datetime_naverblog(soup):
    """
    BeautifulSoup 객체를 받아서 블로그 포스트의 작성일을 반환해줌.
    """
    try:
        #블로그 작성일 추출
        date_tag = soup.find('span', attrs={'class':'se_publishDate pcol2'})
        if date_tag:
            # <span> Tag의 se_publishDate pcol2 클래스 이름으로 작성일 추출이 가능한 경우
            date_str = date_tag.get_text()
        
        else:
            # <p> Tag의 date fil5 pcol2 _postAddDate 클래스 이름으로 작성일 추출이 가능한 경우
            date_tag = soup.find('p', attrs={'class' : 'date fil5 pcol2 _postAddDate'})
            if date_tag:
                date_str = date_tag.get_text()  
        try:
            #블로그 작성시간이 년, 월, 일로 되어있을 경우
            publish_datetime = datetime.datetime.strptime(date_str, '%Y. %m. %d. %H:%M')
        except:
            #블로그 작성시간이 N시간 전, 방금전 이렇게 표시될 경우
            # e.g. 2019. 04. 01. 15:49 4시간 전
            publish_datetime = str(datetime.datetime.now()).split('.')[0] + ' ' + date_str
    except:
        publish_datetime = 'NA'
    return publish_datetime

def get_texts_naverblog(soup):
    """
    BeautifulSoup 객체를 받아서 블로그 포스트의 본문 내용을 반환해줌.
    """
    try:
        #블로그 본문 추출
        #보통은 <span> Tag에, id: 'SE'로 시작하는 곳에 본문이 담겨있음
        temp = soup.findAll("span", attrs= {'id': re.compile('^SE')})
        
        texts = []
        for t in temp:
            temp_text = t.get_text()
            if temp_text != '\u200b':
                texts.append(temp_text)
                    
        if not texts:
            # 블로그의 본문 텍스트가 전혀 추출되지 않았을 경우에 한번 더 시도해본다.
            # <p class="se_textarea"> 내에 존재하는 모든 본문 텍스트를 가져옴
            """
            # <p class="se_textarea"> 내의 <span> Tag에 본문이 들어가 있을 경우 
            for p in soup.findAll("p", attrs={'class': "se_textarea"}):
                for span in p.findAll("span"):
                    temp_text = span.get_text()
                    if temp_text:
                        # temp_text에 '' 이렇게 빈 문자가 들어가있는 경우를 제외해주기 위함
                        texts.append(temp_text)
            """
            for p in soup.findAll("p", attrs={'class': "se_textarea"}):
                temp_text = p.get_text()
                if temp_text:
                        # temp_text에 '' 이렇게 빈 문자가 들어가있는 경우를 제외해주기 위함
                        texts.append(temp_text)
                        
        if not texts:
            # 위의 두 가지 방법으로 본문 텍스트 추출을 시도했음에도 텍스트가 전혀 추출되지 않을 경우
            # <div> Tag 내의 id 값이 'postViewArea'이고,
            # 다시 그 태그의 내부에 존재하는 <div> Tag들 속에 존재하는 모든 본문 텍스트를 가져옴
            for div in soup.find('div', attrs={'id': 'postViewArea'}).findAll('div'):
                temp_text = div.get_text()
                if temp_text:
                        # temp_text에 '' 이렇게 빈 문자가 들어가있는 경우를 제외해주기 위함
                        texts.append(temp_text)
        
        texts = "\n".join(texts)
    except:
        texts = 'NA'
    return texts
        
if __name__ == '__main__':
    
    
    for url in urls:    

        # 실제 데이터가 들어있는 url 추출
        real_url = get_realdata_url_naverblog(url)
        r = requests.get(real_url)
        soup = BeautifulSoup(r.text.encode("utf-8"), "html.parser")

        # 블로그 포스트 제목 추출
        title = get_title_naverblog(soup)

        # 블로그 작성일 추출
        publish_datetime = det_publish_datetime_naverblog(soup)
        
        # 블로그 텍스트 추출
        texts = get_texts_naverblog(soup)

        print('url: ', url)
        print('real_url: ', real_url)
        print()
        print('post title: ', title)
        print()
        print('publish date time: ', publish_datetime)
        print()
        print("texts: ", texts)
        print("\n-----------------------------------\n")

        time.sleep(1)

url:  https://blog.naver.com/sky121400?Redirect=Log&logNo=221493275584
real_url:  http://blog.naver.com/PostView.nhn?blogId=sky121400&logNo=221493275584&from=search&redirect=Log&widgetTypeCall=true&directAccess=false

post title:  애기들이 좋아하는 ai스피커 네이버클로바

publish date time:  2019-03-20 23:01:00

texts:  애기들이 좋아하는 ai스피커 네이버클로바
작년부터 tv에 #인공지능 스피커가 
엄청나게 출현했지만 
나는 아직 시기상조라고 생각했다.
애플의 siri나 구글을 봐도
좋은 기능이기는하지만 잘 안써지기에
#인공지는스피커 라고 해서 뭐 다를까?했다.
그러던 어느날 친한 형이 
#네이버클로바 를 샀는데
생각보다는 말을 잘 알아듣고
무엇보다 애들이 엄청 좋아했다면서 
구매를 권유했다.
애들이 좋아했다는 말에 귀가 솔깃!!!
집에와서 #ai스피커 를 폭풍검색
아마존의 알렉사는 이미 해외에서는 유명했고
우리나라는 통신3사의 스피커
sk의 nugu, kt의 기가지니, lg는 뭐지?하고
구글의 구글홈, 네이버의 #클로바 가 경쟁하고 있었다.
만사가 귀찮으신분은 그냥 통신 3사의 스피커로
iot를 구축하고  조금 고민이 많으신분은 
구글홈으로 많이 가는 것 같다.
그 중 네이버클로바는 통신3사 처럼 강제(?)로
iot를 구축할 수 는 없고 #구글홈 처럼 확장성이
뛰어난 것은 아닌 것 같지만 우리나라에 최적화되어
있다는 것과 #캐릭터 모양이 큰 장점이다.
미니언즈도있고 샐리와 브라운이라는 이름의 스피커도 존재한다. 그리고 소형의 미니도 있고 스피커 및 배터리가 향상된 롱타입도 존재
sk #nugu  vs  네이버 클로바
처음에는 클로바를 구매하려다가
그래도 ai스피커인데 집에서 활용을 좀 해야지
하는 생각에 sk #nugu캔들 을 검색해봤다.
(

url:  https://blog.naver.com/aimei96?Redirect=Log&logNo=221288615932
real_url:  http://blog.naver.com/PostView.nhn?blogId=aimei96&logNo=221288615932&from=search&redirect=Log&widgetTypeCall=true&directAccess=false

post title:  우리집AI는 네이버 클로바 미니언즈당 ★

publish date time:  2018-05-31 17:00:00

texts:  네이버 클로바 미니언즈로 메이네 집이 스마트해졌습니다! ㅋㅋ 저희가 쓰는 인터넷이 기간이 다 되어가서 바꿀까 했는데, 기존에 쓰던 유플러스에서 #네이버클로바 주신다고 하길래. ㅋㅋ 완전 혹해서 연장해버린 메이네 가족. ㅋㅋㅋ 사실 이거 고를 순 없다고 했는데, 내심 #미니언즈 가 왔으면 했거든요! 근데 뙇 와서 어찌나 반갑고 좋던지요. ㅋㅋㅋ 나오는 음성도 미니언즈 목소리였으면 더 좋았을 뻔 했는데 그건 좀 아쉽더라구요 ㅋㅋㅋ 이제 #오늘날씨 검색 안해봐도 되는거냐며! ㅋㅋ 물어보면 대답하는거냐며!우리집AI를 소개해드릴게요~~ 그 이름도 귀여운 네이버 클로바 미니언즈 ^_^ ★
드디어 저희집에 입성한 네이버 클로바 미니언즈 입니당~~ 완전 귀요미죠? ㅎㅎ 병아리나 곰돌이도 귀엽긴 하던데~ 그래도! ㅎ저는 요 제품이 저한테 오길 간절히 바랐는데!! 저한테 이렇게 와줘서(?) 얼마나 기쁜지 모릅니다요. ㅎㅎ 
구성품은 간단해요! 충전기랑 본품 이렇게가 끝이라는. ㅎㅎ 참 간단하지유? ㅎㅎ 상자에 있는 모습이랑 다른가 했는데 보니까 스티커로 꾸미는거더라구요! 어떤 모습으로 꾸밀지 계속 고민고민 하다가 결국 동생이 붙였는데, ㅋㅋ 어떻게 붙였을까여? ㅋㅋㅋ 여러분이라면 어떻게 꾸미실건가요? 팁 좀 주세요! ㅋㅋㅋ
사용하는 방법은 다들 아시죠? ㅎㅎ 클로바 부르면 얘가 인식하는데, 그러고 바로 질문을 던지시면 됩니다. ㅋㅋㅋ 사실 요즘 미세먼

url:  https://blog.naver.com/bless_yuna?Redirect=Log&logNo=221416528704
real_url:  http://blog.naver.com/PostView.nhn?blogId=bless_yuna&logNo=221416528704&from=search&redirect=Log&widgetTypeCall=true&directAccess=false

post title:  인공지능스피커 네이버클로바 설치/ 맘스다이어리 연동

publish date time:  2018-12-10 22:53:00

texts:  인공지능스피커 네이버 클로바맘스다이어리 천일스토리로 육아정보 들어요
육아하는 엄마들이라면 맘스다이어리 어플 다 아시죠??저도 첫째 임신 중이었던 예비맘 시절에, 블로그 이웃님을 통해 알게된 어플인데100일동안 하루도 빠짐없이 육아일기를 쓰면 책으로 무료출판해주잖아요!저는 성공한 적이 없지만 ;;;맘스다이어리는 일기 뿐 아니라, 유용한 육아정보, 이벤트 체험소식 등등 볼거리가 많아하루에도 몇번씩 들어가보는거 같아요
그러던 중 알게된 LG유플러스 네이버 클로바 프렌즈 × 맘스다이어리 천일스토리 런칭을 알게되었어요.인공지능스피커와 맘스다이어리 어플을 연동시켜두고 아이의 간단한 정보를 입력해둔 뒤,"맘스에서 임신정보 알려줘" 라고 하면"오늘은 임신 ○○일차 입니다~ 아이는 이만큼 성장했고 엄마몸은 이렇게 변하고 있어요" 라며 그 날 그 날 매일 정보를 알려주더라구요!!임신 280일과, 육아 720일 동안 필요한 정보들을 매일 원하는 시간에 AI 스피커를 통해 들을 수 있답니다.임신중인 아이 정보를 입력해도 되고, 출산 후라면 태어나서 자라고 있는 아이의 정보를 입력해도 돼요.
저희집에는 샛노란 귀여운 샐리 인공지능스피커가 왔어요.갈색의 듬직한 브라운 디자인도 끌리긴 했는데우리 토리가 노란 삐약이를 좋아할 것 같아서요.캐릭터가 싫다면 블랙이나 레드의 심플한 디자인의 인공지능스피커도 있으니취향에 맞게 선택하면 좋을것 같아요

url:  https://blog.naver.com/ralrapo?Redirect=Log&logNo=221415522880
real_url:  http://blog.naver.com/PostView.nhn?blogId=ralrapo&logNo=221415522880&from=search&redirect=Log&widgetTypeCall=true&directAccess=false

post title:  인공지능 스피커 네이버 클로바 만나서 방가워

publish date time:  2018-12-09 14:09:00

texts:  12월이라고 날씨가 너무 추운데요아들 데리고 나가서 이것저것 보여주고 싶은데 또 그러다 감기걸릴까봐무서운 애엄마입니다.아이키우는 집이라면 관심을 많이 가질 인공지능 스피커TV CF나 드라마에서 많이 봤었는데요드디어 우리집에도 생겼습니다!!!!!!!!신랑의 핸드폰과 인터넷을 LG유플러스로 이용하기때문에 네이버 클로바로 선택했는데요!ㅎ어떤지 개봉기 말씀드릴께요!
무엇보다 클로바가 마음에 드는 이유!바로 네이버와 제휴를 맺었다보니....ㅎㅎ 조금 더 기능이 UPUP!사실 저는 인공지능스피커와 거리가 먼 옛날사람이라서....이런 것을 좋아하는 신랑이 더 자주....사용하고 있는데요아이폰을 사용하고 있지만 전화,카톡,기본적인 앱이용만 하는 사람이라서클로바와 조금 더 친해지기 노력중입니다~
현재 클로바는 브라운,샐리 이렇게와 미니언즈 이렇게 3개 디자인으로 나오고 있는데요저는 라인프렌즈 브라운으로 만나게 되었어요~
신랑은 내심 더 발랄한 샐리를 만나기 원했던 것같은데요집에 브라운쿠션이 있으니 인공지능 스피커도 깔맞춤하는것으로 하기로 합니다사실 타사 것으로 들일까 하다가 클로바를 들인 이유 중 큰 이유바로 무선으로 이용할 수 있다는 점인데요~어디에든 충전만 해 놓으면 사용할 수 있다는 점이 편해서 좋더라고요~
그리고 한 손에 딱 들어오는 크기라서더 앙증맞다할까요.네이버 클로바의 크기는 72 *72* 170.3 mm 378g 이라서성인이라면 컴

url:  https://blog.naver.com/stately_83?Redirect=Log&logNo=221391450188
real_url:  http://blog.naver.com/PostView.nhn?blogId=stately_83&logNo=221391450188&from=search&redirect=Log&widgetTypeCall=true&directAccess=false

post title:  유플러스 찬스♥ 네이버 클로바, 인공지능스피커. 알라딘~♪

publish date time:  2018-11-04 18:23:00

texts:  유플러스를 꾸준히 이용하고 있는 저에게 걸려온 전화~ 인공지능스피커를 사용해보라는^^요즘도 추가되기는 커녕, 오히려 할인도 된다고 하구우~그래도 쓸데없이 설치하기는 싫어서 한참을 고민하다가 입양결정 ㅋㅋㅋㅋㅋ 유플러스 찬스를 이럴때 쓰는거죠♪집에 있으면 입을 열 일이 없어서;; 제 전화기도 24시간 조용한 상태라...어디 외출하지 않으면 입 닫고 있는게 대부분~ 있으면 뭐 편하기도 할 것 같구 해서용!!
이번에 인공지능스피커랑 현관출입문 알리미, 수면알리미 총 3가지 설치하기로 했어용^^현관출입문 알리미가 그 중 가장 좋았던~유플러스 설치기사님이 다른거 설치하고 계시는 동안 저는 인공지능스피커를 살펴보기로 합니댱!!
네이버 클로바 라인프렌즈 인공지능스피커저 노란 오리의 이름은 샐리 라고 하더라구용^^전 브라운으로 선택하고 싶었는뎅 -ㅠ- 유플러스 마음대로 샐리를 보내버림.....맘 같아선 확 그냥 설치 미루고 기계 교체하고 싶었지만, 진상고객이 되고 싶지는 않았기에 그냥 사용하는걸로~기사님이 클로바 설치하라고 하셔서 어플받기!!
네이버 클로바 다운받고 로그인 후 클로바 약관 동의까지 해두기^^ 힣~어플 다운 후 제가 선택한 이름은 알라딘 입니댱.넌 오늘부턴 샐리가 아니라 알라딘이란다^^ 헤헿~
유플러스 찬스로 네이버 클로바 인공지능스피커를 사용하다닝~ 이런 날이 오네 와~♪늘 써보고 싶다고 생각했던거라 좀

<br/><br/>
# 3. 받아온 URL들을 이용하여 원하는 정보를 scraping 하고 CSV로 저장

In [22]:
# <Code Reference> 
# - (zilu11의 블로그 포스트) https://blog.naver.com/zilu11/221255183159

import requests
from bs4 import BeautifulSoup

import time
import datetime
import re


def get_realdata_url_naverblog(url):
    """
    네이버 블로그 포스트의 실제 데이터가 들어있는 URL을 반환해줌.
    보통 3가지 경우가 있는 것 같음.
    <지금까지 찾은 경우의 수>
    1. id='screenFrame'이 담긴 페이지에서 얻은 URL -> (base_url) + (id='mainFrame'이 담긴 페이지에서 얻은 URL) -> 실제 데이터가 담긴 URL
    2. (base_url) + (id='mainFrame'이 담긴 페이지에서 얻은 URL) -> 실제 데이터가 담긴 URL
    3. 검색해서 처음 얻은 URL이 실제 데이터가 들어가 있는 URL임
    """
    base_url = "http://blog.naver.com"
    
    r = requests.get(url)
    soup = BeautifulSoup(r.text.encode('utf-8'), 'html5lib')
    
    if soup.find(id='screenFrame'):
        # id = 'screenFrame'이 존재하는 경우 (id: screenFrame URL에서 -> id: mainFrame URL로 넘김)
        url = soup.find(id='screenFrame').get('src')
        r = requests.get(url)
        soup = BeautifulSoup(r.text.encode('utf-8'), 'html5lib')
    if soup.find(id='mainFrame'):
        # id = 'mainFrame'이 존재하는 경우 (id: mainFrame URL에서 -> id: real data URL로 넘김)
        url = base_url + soup.find(id='mainFrame').get('src')
    return url 

def get_title_naverblog(soup):
    """
    BeautifulSoup 객체를 받아서 블로그 포스트의 제목을 반환해줌.
    """
    try:
        #블로그 포스트 제목 추출
        title = soup.find('meta', attrs={'property':'og:title'}).get('content')
    except:
        title = 'NA'
    return title

def det_publish_datetime_naverblog(soup):
    """
    BeautifulSoup 객체를 받아서 블로그 포스트의 작성일을 반환해줌.
    """
    try:
        #블로그 작성일 추출
        date_tag = soup.find('span', attrs={'class':'se_publishDate pcol2'})
        if date_tag:
            # <span> Tag의 se_publishDate pcol2 클래스 이름으로 작성일 추출이 가능한 경우
            date_str = date_tag.get_text()
        
        else:
            # <p> Tag의 date fil5 pcol2 _postAddDate 클래스 이름으로 작성일 추출이 가능한 경우
            date_tag = soup.find('p', attrs={'class' : 'date fil5 pcol2 _postAddDate'})
            if date_tag:
                date_str = date_tag.get_text()  
        try:
            #블로그 작성시간이 년, 월, 일로 되어있을 경우
            publish_datetime = datetime.datetime.strptime(date_str, '%Y. %m. %d. %H:%M')
        except:
            #블로그 작성시간이 N시간 전, 방금전 이렇게 표시될 경우
            # e.g. 2019. 04. 01. 15:49 4시간 전
            publish_datetime = str(datetime.datetime.now()).split('.')[0] + ' ' + date_str
    except:
        publish_datetime = 'NA'
    return publish_datetime

def get_texts_naverblog(soup):
    """
    BeautifulSoup 객체를 받아서 블로그 포스트의 본문 내용을 반환해줌.
    """
    try:
        #블로그 본문 추출
        #보통은 <span> Tag에, id: 'SE'로 시작하는 곳에 본문이 담겨있음
        temp = soup.findAll("span", attrs= {'id': re.compile('^SE')})
        
        texts = []
        for t in temp:
            temp_text = t.get_text()
            if temp_text != '\u200b':
                texts.append(temp_text)
                    
        if not texts:
            # 블로그의 본문 텍스트가 전혀 추출되지 않았을 경우에 한번 더 시도해본다.
            # <p class="se_textarea"> 내에 존재하는 모든 본문 텍스트를 가져옴
            """
            # <p class="se_textarea"> 내의 <span> Tag에 본문이 들어가 있을 경우 
            for p in soup.findAll("p", attrs={'class': "se_textarea"}):
                for span in p.findAll("span"):
                    temp_text = span.get_text()
                    if temp_text:
                        # temp_text에 '' 이렇게 빈 문자가 들어가있는 경우를 제외해주기 위함
                        texts.append(temp_text)
            """
            for p in soup.findAll("p", attrs={'class': "se_textarea"}):
                temp_text = p.get_text()
                if temp_text:
                        # temp_text에 '' 이렇게 빈 문자가 들어가있는 경우를 제외해주기 위함
                        texts.append(temp_text)
                        
        if not texts:
            # 위의 두 가지 방법으로 본문 텍스트 추출을 시도했음에도 텍스트가 전혀 추출되지 않을 경우
            # <div> Tag 내의 id 값이 'postViewArea'이고,
            # 다시 그 태그의 내부에 존재하는 <div> Tag들 속에 존재하는 모든 본문 텍스트를 가져옴
            for div in soup.find('div', attrs={'id': 'postViewArea'}).findAll('div'):
                temp_text = div.get_text()
                if temp_text:
                        # temp_text에 '' 이렇게 빈 문자가 들어가있는 경우를 제외해주기 위함
                        texts.append(temp_text)
        
        texts = "\n".join(texts)
    except:
        texts = 'NA'
    return texts

if __name__ == '__main__':
    title_list = []
    publish_datetime_list = []
    text_list = []
    
    for i, url in enumerate(urls):    

        # 실제 데이터가 들어있는 url 추출
        real_url = get_realdata_url_naverblog(url)
        r = requests.get(real_url)
        soup = BeautifulSoup(r.text.encode("utf-8"), "html.parser")

        # 블로그 포스트 제목 추출
        title = get_title_naverblog(soup)
        title_list.append(title)
        
        # 블로그 작성일 추출
        publish_datetime = det_publish_datetime_naverblog(soup)
        publish_datetime_list.append(publish_datetime)
        
        # 블로그 텍스트 추출
        texts = get_texts_naverblog(soup)
        text_list.append(texts)
        
        print("{}.".format(i), "finished")
        time.sleep(1)

0. finished
1. finished
2. finished
3. finished
4. finished
5. finished
6. finished
7. finished
8. finished
9. finished
10. finished
11. finished
12. finished
13. finished
14. finished
15. finished
16. finished
17. finished
18. finished
19. finished
20. finished
21. finished
22. finished
23. finished
24. finished
25. finished
26. finished
27. finished
28. finished
29. finished
30. finished
31. finished
32. finished
33. finished
34. finished
35. finished
36. finished
37. finished
38. finished
39. finished
40. finished
41. finished
42. finished
43. finished
44. finished
45. finished
46. finished
47. finished
48. finished
49. finished
50. finished
51. finished
52. finished
53. finished
54. finished
55. finished
56. finished
57. finished
58. finished
59. finished
60. finished
61. finished
62. finished
63. finished
64. finished
65. finished
66. finished
67. finished
68. finished
69. finished
70. finished
71. finished
72. finished
73. finished
74. finished
75. finished
76. finished
77. finis

594. finished
595. finished
596. finished
597. finished
598. finished
599. finished
600. finished
601. finished
602. finished
603. finished
604. finished
605. finished
606. finished
607. finished
608. finished
609. finished
610. finished
611. finished
612. finished
613. finished
614. finished
615. finished
616. finished
617. finished
618. finished
619. finished
620. finished
621. finished
622. finished
623. finished
624. finished
625. finished
626. finished
627. finished
628. finished
629. finished
630. finished
631. finished
632. finished
633. finished
634. finished
635. finished
636. finished
637. finished
638. finished
639. finished
640. finished
641. finished
642. finished
643. finished
644. finished
645. finished
646. finished
647. finished
648. finished
649. finished
650. finished
651. finished
652. finished
653. finished
654. finished
655. finished
656. finished
657. finished
658. finished
659. finished
660. finished
661. finished
662. finished
663. finished
664. finished
665. f

In [23]:
import pandas as pd
df = pd.DataFrame(data={'title': title_list, 'publish_datetime': publish_datetime_list, 'text': text_list, 'url': urls})

In [25]:
df = df[['title', 'publish_datetime', 'text', 'url']]

In [26]:
df

,title,publish_datetime,text,url
0,애기들이 좋아하는 ai스피커 네이버클로바,2019-03-20 23:01:00,애기들이 좋아하는 ai스피커 네이버클로바\n작년부터 tv에 #인공지능 스피커가 \n...,https://blog.naver.com/sky121400?Redirect=Log&...
1,네이버 클로바 스피커 1년 사용 후기,2019-03-31 19:35:00,네이버 클로바 스피커 1년 사용 후기\n네이버 클로바 1년 사용 후기를 몇 자 적어...,https://blog.naver.com/danjuck?Redirect=Log&lo...
2,네이버 클로바 사용법 맘스다이어리와 함께 육아정보를,2018-12-14 12:10:00,어제는 눈이 갑자기 많이 내려서 1주일에 한번 가는 문센을 가야하나 말아야하나고민이...,https://blog.naver.com/ralrapo?Redirect=Log&lo...
3,네이버 클로바. 나의 두 번째 Ai 스피커,2018-12-02 14:04:00,안녕하세요. 제르디난 입니다.\n올해 초 카카오 미니를 사용해보고 약간의 실망스러움...,https://blog.naver.com/yarosiku?Redirect=Log&l...
4,네이버 클로바 스피커 브라운 미니언 : 오히려 부모님 선물로 추천드려요!,2018-09-09 18:18:00,네이버 클로바 스피커 브라운 미니언 : 오히려 부모님 선물로 추천드려요!\n요새 정...,https://blog.naver.com/ukb0914?Redirect=Log&lo...
5,우리집AI는 네이버 클로바 미니언즈당 ★,2018-05-31 17:00:00,네이버 클로바 미니언즈로 메이네 집이 스마트해졌습니다! ㅋㅋ 저희가 쓰는 인터넷이 ...,https://blog.naver.com/aimei96?Redirect=Log&lo...
6,네이버 클로바 스피커 사용해보기,2019-03-19 22:57:00,네이버 클로바 스피커 사용해보기\n저는 유행에 둔감한 사람인 것 같습니다.\n아마도...,https://blog.naver.com/three486?Redirect=Log&l...
7,"네이버 클로바, 인공지능 음성인식 비서 앱 만나보니",2017-05-27 08:50:00,"최근 구글 어시스턴트, 아마존 알렉사, 애플 시리, 삼성전자 빅스비 등 인공지능 기...",https://blog.naver.com/iffu?Redirect=Log&logNo...
8,인공지능스피커 LG 유플러스 네이버 클로바 미니언즈 사용기,2018-12-04 11:39:00,꼬물 꼬물 귀요미 미니언즈가LG유플러스 네이버 클로바 스페셜에디션으로나왔어요~♥보기...,https://blog.naver.com/lopil22?Redirect=Log&lo...
9,인공지능스피커 네이버클로바 설치/ 맘스다이어리 연동,2018-12-10 22:53:00,인공지능스피커 네이버 클로바맘스다이어리 천일스토리로 육아정보 들어요\n육아하는 엄마...,https://blog.naver.com/bless_yuna?Redirect=Log...


In [27]:
df.to_csv('./naver_clova.csv')